In [45]:
import random as random
import Bio
import numpy as np
import sys
import os
import networkx as nx
import random as random
from tqdm import tqdm # loading bar
from utils_copy import linear_C, get_cost_2, get_sequence_string, parse_fasta_multiple, create_score_matrix, write_alignments_to_file, linear_backtrack, fill_graph,new_sp_approxi_combi
from utils_copy import convert_to_desired_format_nr_version, compute_cost, my_traversal_simply, extend_alignment_chaos, find_min_span_edges_testing, parse_fasta_multiple_remove_n
import timeit
from utils_copy import al_integrity_testt
from old_for_testing.sp_approx import sp_approx
from old_for_testing.utils import *
from utils_copy import sum_of_column


score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5

In [4]:
def make_MSA_position_overview_dict(res_matrix):#initializes the overview dictionary
    in_which_MSA_is_it={}
    names= np.unique(res_matrix[:, 2:])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    print(in_which_MSA_is_it)


In [5]:
def find_highest_second_element(d, first_element): #to find the highest col nr already in the MSA
    highest_second_element = 0
    for key, value in d.items():
        if value[0] == first_element:
            second_element = value[1]
            if second_element > highest_second_element:
                highest_second_element = second_element
    return highest_second_element

In [6]:
def update_MSA_position_overview_dict(MSA_position_overview_dict,node1,node2): #may be somehow inadequate!
    print(node1,node2)
    node1_origin_MSA=MSA_position_overview_dict[node1][0]
    node2_origin_MSA=MSA_position_overview_dict[node2][0]
    node1_origin_MSA_col=MSA_position_overview_dict[node1][1]
    node2_origin_col=MSA_position_overview_dict[node2][1]
    #now I have the original positions in from the position overview dict!
    nr_we_got_to=find_highest_second_element(MSA_position_overview_dict,MSA_position_overview_dict[node1])
    MSA_position_overview_dict[node2]=[node1_origin_MSA,(nr_we_got_to+1)] #update the pos_dict to show that node2 changed MSA and make the col one bigger than the max
    return MSA_position_overview_dict

In [7]:
def get_MSA_pos(MSA_position_overview_dict, node):
    value=MSA_position_overview_dict[node]
    MSA_pos=value[0]
    MSA_pos_col=value[1]
    return MSA_pos,MSA_pos_col


In [8]:
def finish_running_through_MSA1(MSA1,MSA2,j,i,l):
    how_many_cols_in_MSA2=len(MSA2[0])
    gaps_to_add=['-']*how_many_cols_in_MSA2
    new_col=list(MSA1[j])+gaps_to_add
    return new_col

def finish_running_through_MSA2(MSA1,MSA2,j,i,l):
    how_many_cols_in_MSA1=len(MSA1[0])
    gaps_to_prepend=['-']*how_many_cols_in_MSA1
    new_col=gaps_to_prepend+list(MSA2[l])
    return new_col

In [9]:
def equality_here_test(a,b):
    equality=False
    if a=='-' and b=='-' or a!='-' and b!='-':
        equality=True
    return equality

In [10]:
def alt_handle_case1(MSA1,MSA2,i,j,l):
    how_many_cols_in_MSA1=len(MSA1[0])
    gaps_to_prepend=['-']*how_many_cols_in_MSA1
    new_col=gaps_to_prepend+list(MSA2[l])
    return new_col #after this, increase only l
def alt_handle_case2(MSA1,MSA2,i,j,l):
    how_many_cols_in_MSA2=len(MSA2[0])
    gaps_to_append=['-']*how_many_cols_in_MSA2
    col_now=MSA1[j]
    col_new=list(MSA1[j])+gaps_to_append
    return col_new #after this, increase only j
def alt_handle_case3(MSA1,MSA2,i,j,l):
    col_new=list(MSA1[j])+list(MSA2[l])
    return col_new #after this, increase i, j and l
def alt_handle_case4(MSA1,MSA2,i,j,l):
    col_new=list(MSA1[j])+list(MSA2[l])
    return col_new #after this, increase j and l, but not 

In [11]:
def new_identify_merge_case_compact(guide_PA,MSA1,MSA2,node1,node2,MSA_position_overview_dict,i,j,l):
    node1_position=MSA_position_overview_dict[node1]
    node1_col=node1_position[1]
    node2_position=MSA_position_overview_dict[node2]
    node2_col=node2_position[1]
    case=None
    move_on_in_guide_flag=False
    a=guide_PA[i][0]
    b=guide_PA[i][1]
    c= MSA1[j][node1_col]
    d=MSA2[l][node2_col]

    if equality_here_test(a,c)==True and equality_here_test(b,d)==True: case=3 #a==c and b==d
    elif equality_here_test(a,c)==False and equality_here_test(b,d)==False: case=4 #a!=c and b!=d
    elif equality_here_test(a,c)==True:#a==c
        if d!='-':
            case= 2
            move_on_in_guide_flag=True
        else: case= 1
    elif equality_here_test(a,c)==False: #a!=c
        if a=='-':
            case= 1
            move_on_in_guide_flag=True
        else: case=2
    else: sys.exit()
    return case,move_on_in_guide_flag

In [12]:
move_on_in_guide_flag=False
case=False
a='a'
b='a'
c='a'
d='a'
if equality_here_test(a,c)==True and equality_here_test(b,d)==True: case=3 #a==c and b==d
elif equality_here_test(a,c)==False and equality_here_test(b,d)==False: case=4 #a!=c and b!=d
elif equality_here_test(a,c)==True:#a==c
    if d!='-':
        case= 2
        move_on_in_guide_flag=True
    else: case= 1
elif equality_here_test(a,c)==False: #a!=c
    if a=='-':
        case= 1
        move_on_in_guide_flag=True
    else: case=2
else: sys.exit()
print (case,move_on_in_guide_flag)



3 False


In [52]:
def alt_alt_identify_merge_case(guide_PA,MSA1,MSA2,node1,node2,MSA_position_overview_dict,i,j,l):
    node1_position=MSA_position_overview_dict[node1]
    node1_col=node1_position[1]
    node2_position=MSA_position_overview_dict[node2]
    node2_col=node2_position[1]
    case=None
    move_on_in_guide_flag=False
    if equality_here_test(guide_PA[i][0],MSA1[j][node1_col])==True and equality_here_test(guide_PA[i][1],MSA2[l][node2_col])==True:
        case=3
    #elif equality_here_test(guide_PA[i][0],MSA1[j][node1_col])==False and equality_here_test(guide_PA[i][1],MSA2[l][node2_col]==False):
    elif equality_here_test(guide_PA[i][0], MSA1[j][node1_col]) == False and equality_here_test(guide_PA[i][1], MSA2[l][node2_col]) == False:
        case=4
    elif guide_PA[i][0]=='-' and MSA1[j][node1_col]=='-':
        case=1
        print(str(guide_PA[i][0])," , "+str(guide_PA[i][1])+" is g1 and g2!")
        if guide_PA[i][0]=='-' and equality_here_test(guide_PA[i][1],MSA1[j][node1_col])==False: #test for flag in case1 new
            move_on_in_guide_flag=True
    elif guide_PA[i][1]=='-' and MSA2[l][node2_col]=='-':
        case=2
        print(str(guide_PA[i][0])," , "+str(guide_PA[i][1])+" is g1 and g2!")
        if guide_PA[i][1]=='-' and equality_here_test(guide_PA[i][1],MSA2[l][node2_col])==False: #test for flag in case2
            move_on_in_guide_flag=True
    elif guide_PA[i][0]!='-' and MSA1[j][node1_col]!='-' and MSA2[l][node2_col]=='-':
        case=1
        print(str(guide_PA[i][0])," , "+str(guide_PA[i][1])+" is g1 and g2!")
        if guide_PA[i][0]=='-' and equality_here_test(guide_PA[i][1],MSA1[j][node1_col])==False: #test for flag in case1 new
            move_on_in_guide_flag=True
    elif guide_PA[i][0]!='-' and MSA1[j][node1_col]!='-' and guide_PA[i][1]=='-':
        case=2
        print(str(guide_PA[i][0])," , "+str(guide_PA[i][1])+" is g1 and g2!")
        if guide_PA[i][1]=='-' and equality_here_test(guide_PA[i][1],MSA2[l][node2_col])==False: #test for flag in case2
            move_on_in_guide_flag=True
    elif guide_PA[i][1]!='-' and MSA2[l][node2_col]!='-' and guide_PA[i][0]=='-':
        case=1
        print(str(guide_PA[i][0])," , "+str(guide_PA[i][1])+" is g1 and g2!")
        if guide_PA[i][0]=='-' and equality_here_test(guide_PA[i][1],MSA1[j][node1_col])==False: #test for flag in case1 new
            move_on_in_guide_flag=True
    elif guide_PA[i][1]!='-' and MSA2[l][node2_col]!='-' and MSA1[j][node1_col]=='-':
        case=2
        print(str(guide_PA[i][0])," , "+str(guide_PA[i][1])+" is g1 and g2!")
        if guide_PA[i][1]=='-' and equality_here_test(guide_PA[i][1],MSA2[l][node2_col])==False: #test for flag in case22
            move_on_in_guide_flag=True
    else:
        sys.exit()
    return case,move_on_in_guide_flag

    
def alt_alt_merge_united(guide_PA,MSA_list,in_which_MSA_is_it,node1,node2):
    print(type(in_which_MSA_is_it))
    print(in_which_MSA_is_it)
    print(type(node1))
    print(type(node2))
    MSA_of_node1=in_which_MSA_is_it[node1][0]
    MSA_of_node2=in_which_MSA_is_it[node2][0]
    spot_of_node1=in_which_MSA_is_it[node1][1]
    spot_of_node2=in_which_MSA_is_it[node2][1]
    print(MSA_of_node1)
    print(MSA_of_node2)
    MSA1=MSA_list[int(MSA_of_node1)]
    MSA2=MSA_list[int(MSA_of_node2)]
    
    j=0 
    i=0
    l=0
    MSA_new=[]
    while i<=(len(guide_PA)-1) and j<=(len(MSA1)-1) and l<=(len(MSA2)-1):
        print("I don't have the case identified yet, but MSA1 is: " + str(MSA1[j]) + " MSA2 is: " + str(MSA2[l]) + " ,and the guide is: " + str(guide_PA[i]))
        print("nodes pointing to cols: " + str(spot_of_node1) + " , " + str(spot_of_node2))
        print("i,j,l: "+str(i)+','+str(j)+','+str(l))
        print("max for those should be: "+str(len(guide_PA))+','+str(len(MSA1))+','+str(len(MSA2)))
        case,move_on_in_guide_flag=new_identify_merge_case_compact(guide_PA,MSA1,MSA2,node1,node2, in_which_MSA_is_it,i,j,l)
        print("flag: "+str(move_on_in_guide_flag))
        print("case:"+str(case))
        if case==1:
            col=alt_handle_case1(MSA1,MSA2,i,j,l)
            MSA_new.append(col) 
            if move_on_in_guide_flag==True:
                print("the damn flag was true!")
                i+=1
            l+=1
        if case==2:
            col=alt_handle_case2(MSA1,MSA2,i,j,l)
            MSA_new.append(col)
            if move_on_in_guide_flag==True:
                print("the flag was true dammit")
                i+=1
            j+=1
        if case==3:
            col=alt_handle_case3(MSA1,MSA2,i,j,l)
            MSA_new.append(col)
            i+=1
            j+=1
            l+=1
        if case==4:
            col=alt_handle_case4(MSA1,MSA2,i,j,l)
            MSA_new.append(col)
            j+=1
            l+=1
    if j <= (len(MSA1)-1):
        while j<= (len(MSA1)-1):
            column=finish_running_through_MSA1(MSA1,MSA2,j,i,l)
            j+=1
            #l+=1
            MSA_new.append(column)
    if l<= (len(MSA2)-1):
        while l<= (len(MSA2)-1):
            column=finish_running_through_MSA2(MSA1,MSA2,j,i,l)
            l+=1
            MSA_new.append(column)
    if i<=(len(guide_PA)-1):
        print('yikes, the strings have run out but the guide has not')
        i+=1
    return MSA_new


def alt_alt_the_whole_thing(seqs,score_matrix,gapcost):
    # Make a matrix to hold pairwise alignment costs for all alignment combinations!
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # Loop over all pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2))
        print("Here comes the distance matrix produced by the alignments: \n")
        print(matrix)
    matrix_for_MST=matrix #copy the matrix, so that we can keep the old matrix and make a changed version to the "pseudomatrix" version
    matrix_for_MST=convert_to_desired_format_nr_version(matrix_for_MST) #making the "pseudomatrix"
    print("matrix for MST: "+str(matrix_for_MST))
    min_span_edges_res=find_min_span_edges_testing(matrix_for_MST)
    print("min span edges: "+str(min_span_edges_res))
    in_which_MSA_is_it={}
    names= np.unique(min_span_edges_res[:, 2:])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    #MSA_list=[[*seq] for seq in seqs] #initalizing it as every string in their own alignment with nobody else :) 
    MSA_list=[[[char] for char in seq] for seq in seqs] #let's try this instead!
    print("here comes the pos-dict: "+str(in_which_MSA_is_it))
    print("look at the current MSA-list: "+str(MSA_list))
    who_aligned_to_who=[]
    for row in min_span_edges_res:
        if row[0]=="*":
            node1 = row[2]  # Extract the first node
            node2 = row[3]  # Extract the second node
            who_aligned_to_who.append([node1,node2])
            cost=linear_C(gap_cost,score_matrix,seqs[int(node1)],seqs[int(node2)]) #doing the alignment of the two strings :) 
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(node1)], seqs[int(node2)], cost, score_matrix, gap_cost)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            A = [list(e) for e in zip(alignment1,alignment2)]
            orig_str_node2=seqs[int(node2)]
            united_MSA_new=alt_alt_merge_united(A,MSA_list,in_which_MSA_is_it,node1,node2)#should bring the merged alignment. Just need to update MSA_list and the dict now then...but also for the others that were joined into it now......
            print("this is the union produced: "+ str(united_MSA_new))
            which_spot_in_MSA_list_to_update=in_which_MSA_is_it[node1][0]
            which_spot_in_MSA_list_to_remove=in_which_MSA_is_it[node2][0]
            #in_which_MSA_is_it=update_MSA_position_overview_dict(in_which_MSA_is_it,node1,node2)
            
            MSA_list[which_spot_in_MSA_list_to_update]=united_MSA_new
            MSA_list.pop(which_spot_in_MSA_list_to_remove)
            #now to keep track of where everything is, a nightmare..
            companions_list=[] #to keep the keys of the strings already in MSA2 for later update.
            for key, value in in_which_MSA_is_it.items():
                how_many_cols_already_in_MS1=find_highest_second_element(in_which_MSA_is_it,which_spot_in_MSA_list_to_update)
                if value[0]>which_spot_in_MSA_list_to_remove:
                    value[0]-=1
                elif value[0]==which_spot_in_MSA_list_to_remove:
                    value[0]=which_spot_in_MSA_list_to_update
                    companions_list.append(key)
            print("this is how many cols already in MSA1: "+str(how_many_cols_already_in_MS1))
            print("this is the companions list: "+str(companions_list))
            for element in companions_list:
                col_of_element_in_old_MSA2=in_which_MSA_is_it[element][1]
                print("this is column nr of node1 in old MSA2: "+str(col_of_element_in_old_MSA2))
                col_in_new_MSA_node2=int(how_many_cols_already_in_MS1)+int(col_of_element_in_old_MSA2)+1
                print("new col nr of node2: "+str(col_in_new_MSA_node2))
                print("this is the place I try to change: "+ str(in_which_MSA_is_it[element][1]))
                in_which_MSA_is_it[element][1]=col_in_new_MSA_node2
                print("this is the updated dict: "+str(in_which_MSA_is_it))
        print(MSA_list)
    print("here comes the full MSA_list: ")
    print(MSA_list)
    #integrity check part 1, to check if each string is the same before and after, except for gaps.
    for i,seq in enumerate(seqs):
        col_to_extract=in_which_MSA_is_it[str(i)][1]
        j=0
        new_str_with_gaps=[]
        new_str_no_gaps=[]
        while j<=len(MSA_list[0])-1:
            found=MSA_list[0][j][col_to_extract]
            j+=1
            new_str_with_gaps.append(found)
            if found !='-':
                new_str_no_gaps.append(found)
        new_str_no_gaps=''.join(new_str_no_gaps)
        new_str_with_gaps=''.join(new_str_with_gaps)
        if new_str_no_gaps==seq:
            print("integrity check 1 passed for seq "+str(i))
        else:
            print("Yikes, integrity check 1 did not pas for seq "+str(i)+". constrast: \n"+str(new_str_no_gaps)+"\n"+str(seq))
        #part 2 lol, are the alignments preserved, expect for gaps???
    for element in who_aligned_to_who:
        seq1_nr=element[0]
        seq2_nr=element[1]
        pos_in_MSA_seq1=in_which_MSA_is_it[seq1_nr][1]
        pos_in_MSA_seq2=in_which_MSA_is_it[seq2_nr][1]
        seq1_from_MSA=[]
        seq2_from_MSA=[]
        j=0
        while j<=len(MSA_list[0])-1:
            found=MSA_list[0][j][pos_in_MSA_seq1]
            seq1_from_MSA.append(found)
            j+=1
        j=0
        while j<=len(MSA_list[0])-1:
            found=MSA_list[0][j][pos_in_MSA_seq2]
            seq2_from_MSA.append(found)
            j+=1
        union=[]
        k=0
        while k<=len(seq2_from_MSA)-1:
            el1=seq1_from_MSA[k]
            el2=seq2_from_MSA[k]
            tuple_like_zip=[el1,el2]
            union.append(tuple_like_zip)
            k+=1
        cost_after_MSA=compute_cost(union,score_matrix,gap_cost)
        if cost_after_MSA==matrix[int(seq1_nr)][int(seq2_nr)]:
            print("integrity test 2 passed for: "+str(seq1_nr)+" and "+ str(seq2_nr))
        else:
            print("Yikes, integrity check 2 did not pas for: "+str(seq1_nr)+" and "+ str(seq2_nr))
            print("Costs were before and after:"+str(matrix[int(seq1_nr)][int(seq2_nr)]))
    total_cost = compute_cost(MSA_list[0], score_matrix, gap_cost)
    return(total_cost,MSA_list)

            

        

In [54]:
seqs_for_small_test=['cgcgc','gtaa','atacg']
brca_seqs_work_please,names=parse_fasta_multiple('brca1-testseqs.fasta')
gap_cost=5
alt_alt_the_whole_thing(brca_seqs_work_please,score_matrix,gap_cost)
#alt_alt_the_whole_thing(seqs_for_small_test,score_matrix,gap_cost)

Here comes the distance matrix produced by the alignments: 

[[  0.  18. 384. 457.  38. 463.  50. 464.]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]]
Here comes the distance matrix produced by the alignments: 

[[  0.  18. 384. 457.  38. 463.  50. 464.]
 [ 18.   0. 385. 455.  44. 455.  56. 469.]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan  nan  nan]]
Here comes the distance matrix produced by the alignments: 

[[  0.  18. 384. 457.  38. 463.  50. 464.]
 [ 18.   0. 385. 455.  44. 455.  56. 469.]
 [384. 385.   0. 440. 385. 455. 391. 354.

(15047,
 [[['-', '-', '-', '-', '-', 'c', '-', '-'],
   ['a', 'a', 'a', 'a', 'g', 'g', 'g', 'g'],
   ['t', 't', 't', 't', 'c', 'c', 't', 't'],
   ['g', 'g', 'g', 'g', 'g', '-', '-', '-'],
   ['g', 'g', 'g', 'g', 'a', 't', 'a', '-'],
   ['a', 'a', 'a', 'a', 'a', 'g', 'c', 't'],
   ['t', 't', 't', 't', '-', '-', '-', '-'],
   ['t', 't', 't', 't', '-', '-', '-', '-'],
   ['t', 't', 't', 't', '-', '-', '-', '-'],
   ['a', 'a', 'a', 'a', 'a', 'g', 'c', 'c'],
   ['t', 't', 't', 't', '-', '-', '-', '-'],
   ['c', 'c', 'c', 'c', '-', '-', '-', '-'],
   ['t', 't', 't', 't', 't', 't', 't', '-'],
   ['g', 'g', 'g', 'g', 'g', 'g', '-', '-'],
   ['c', 'c', 'c', 'c', 't', 'c', 't', 'c'],
   ['g', 'g', 't', 't', '-', '-', '-', '-'],
   ['g', 'g', 'g', 'c', 'a', 'a', 'g', 'g'],
   ['a', 'a', 't', 't', 'a', 'a', 'a', 'a'],
   ['t', 't', 't', 't', '-', '-', '-', '-'],
   ['c', 'c', 'c', 'c', 'c', 'c', 't', '-'],
   ['a', 'g', 'g', 'g', 'a', 't', 't', '-'],
   ['t', 't', 'c', 'c', 'c', 'c', 't', '-'],
  